In [1]:
import os
import cv2
from shutil import copyfile
import matplotlib.pyplot as plt
import csv
import random

In [2]:
img_root = "/media/arg_ws3/5E703E3A703E18EB/data/MM_real_data/img/"
seg_root = "/media/arg_ws3/5E703E3A703E18EB/data/MM_real_data/obj_mask/"
fcn_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/"
obj_list = os.listdir(img_root)

In [3]:
os.listdir(img_root)

['3m',
 'andes',
 'cocacola',
 'crayola',
 'folgers',
 'heineken',
 'hunts',
 'kellogg',
 'kleenex',
 'kotex',
 'libava',
 'macadamia',
 'milo',
 'mm',
 'pocky',
 'raisins',
 'stax',
 'swissmiss',
 'vanish',
 'viva']

In [4]:
for obj in obj_list:
    print(obj)
    scene_list = os.listdir(os.path.join(img_root, obj))
    for scene in scene_list:
        data_list = os.listdir(os.path.join(img_root, obj, scene))
        for data in data_list:
            if ".jpg" in data:
                img_path = os.path.join(img_root, obj, scene, data)
                seg_name = data.split(".jpg")
                seg_name = seg_name[0] + ".png"
                seg_path = os.path.join(seg_root, obj, scene, seg_name)
                # copyfile(src, dst)
                #if not os.path.isfile(img_path):
                #    print('1')
                if not os.path.isfile(seg_path):
                    seg_name = data
                    seg_path = os.path.join(seg_root, obj, scene, seg_name)
                    if not os.path.isfile(seg_path):
                        print("WTF, someting wrong!!!!")
                        continue
                copyfile(img_path, os.path.join(fcn_root, "image", obj+"_"+scene+"_"+data))
                copyfile(seg_path, os.path.join(fcn_root, "seg", obj+"_"+scene+"_"+seg_name))

3m
andes
cocacola
crayola
folgers
heineken
hunts
kellogg
kleenex
kotex
libava
macadamia
milo
mm
pocky
raisins
stax
swissmiss
vanish
viva


In [15]:
seg_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/seg/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/mask/"
seg_list = os.listdir(seg_root)
for seg in seg_list:
    obj_class = seg.split('_scene')[0]
    mask_name = seg.split('.')[0] + '.png'
    if obj_class in obj_list:
        class_idx = obj_list.index(obj_class) + 1
        seg_path = os.path.join(seg_root, seg)
        img = cv2.imread(seg_path, cv2.IMREAD_UNCHANGED)
        img[img > 0] = class_idx
        if len(img.shape)==3:
            img = img[:,:,2]
        cv2.imwrite(mask_root + mask_name, img)
    else:
        print('wrong')

# Generate csv file

In [16]:
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/image/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/mask/"
image_list = os.listdir(image_root)
random.shuffle(image_list)
ratio = 0.9
training_list = []
test_list = []
count = 0
for image in image_list:
    if count < len(image_list)*ratio:
        training_list.append(image)
    else:
        test_list.append(image)
    count = count + 1

In [17]:
with open(fcn_root + 'train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for image in training_list:
        mask_name = image.split(".jpg")[0] + ".png"
        writer.writerow(["image/" + image, "mask/" + mask_name])
with open(fcn_root + 'test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for image in test_list:
        mask_name = image.split(".jpg")[0] + ".png"
        writer.writerow(["image/" + image, "mask/" + mask_name])